In [10]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object, save_object
from detectron.datasets.roidb import combined_roidb_for_training
import detectron.utils.vis as vis_utils
from detectron.core.config import cfg


def checkMkdir(dirname):
    if not osp.isdir(dirname):
        os.makedirs(dirname)
        
outValFolder = '/media/data/chnxi/GOD/json_annotations/val'

checkMkdir(outValFolder)
        
datasetList = ['COCO_91', 'HomeFurniture',  'OpenImage'] # ['FashionV2'] # #, 

godLabelFile = '/media/data/chnxi/GOD/taxonomy/GOD_taxonomy_V1.tsv'

godLabels = [l.rstrip().split('\t') for l in open(godLabelFile,'r').readlines()[1:]]
print (godLabels[:10])

mappingFormat = '/media/data/chnxi/GOD/mappings/dataset_nameid_to_GOD_nameid/{}_mapping_to_GOD.pkl'

dataPath = '/media/data/chnxi/'
valSetFiles = {'COCO_91': dataPath + 'coco/annotations/instances_minival2014.json',
                'FashionV2': dataPath + 'FashionV2/json_annotations/val_annotations.json',
                'HomeFurniture': dataPath + 'HomeFurniture/json_annotations/val_annotations.json',
                'OpenImage': dataPath + 'OpenImage/json_annotations/validation_annotations.json'
               }

[['animal\\amphibian\\frog', '1'], ['animal\\arthropod\\arachnid\\scorpion', '2'], ['animal\\arthropod\\arachnid\\spider', '3'], ['animal\\arthropod\\crustacean\\crab', '4'], ['animal\\arthropod\\crustacean\\lobster', '5'], ['animal\\arthropod\\crustacean\\shrimp', '6'], ['animal\\arthropod\\insect\\ant', '7'], ['animal\\arthropod\\insect\\bee', '8'], ['animal\\arthropod\\insect\\butterfly', '9'], ['animal\\arthropod\\insect\\caterpillar', '10']]


In [11]:
print (mapping['name_to_god_name'])

{'full_body-suit': 'fashion\\full_body\\suit', 'accessories-gloves': 'fashion\\gloves', 'bottom_wear-cargo-pants': 'fashion\\bottom_wear\\cargo_pants', 'dresses-wedding': 'fashion\\dresses\\wedding_dress', 'shoes-wedges': 'fashion\\shoes\\wedges', 'bag-duffel': 'fashion\\bag\\duffel_bag', 'swim_wear-cover-up': 'fashion\\swim_wear\\cover-up', 'bottom_wear-jeans': 'fashion\\bottom_wear\\jeans', 'outer_wear-coat': 'fashion\\outer_wear\\coat', 'bag-tote': 'fashion\\bag\\tote', 'accessories-glasses': 'fashion\\glasses', 'full_body-overalls': 'fashion\\full_body\\overalls', 'dresses-cocktail': 'fashion\\dresses\\cocktail_dress', 'top_wear-t-shirt': 'fashion\\top_wear\\t_shirt', 'top_wear-short-sleeve-button': 'fashion\\top_wear\\short_sleeve_button', 'under_wear-tights': 'fashion\\under_wear\\tights', 'bottom_wear-cargo-shorts': 'fashion\\bottom_wear\\cargo_shorts', 'top_wear-strapless': 'fashion\\top_wear\\strapless', 'accessories-hat': 'fashion\\head_wear\\hat', 'accessories-scarf': 'fashi

In [14]:
updateAnno = True
for dataset in datasetList:
    print ("dataset = {}".format(dataset))
    outFileName = osp.join(outValFolder, '{}_val_annotations.json'.format(dataset))
    print ("outFileName = {}".format(outFileName))
    mapFile = mappingFormat.format(dataset)
    mapping = pkl.load(open(mapFile,'rb'))
    #print (mapping.keys())
    id_map = mapping['id_map']
    name_map = mapping['name_to_god_name']
    
    annoFile = valSetFiles[dataset]
    annoData = json.load(open(annoFile,'r'))
    #print (anno.keys())
    
    
    godCateg = []
    supercat = 'GOD_{}'.format(dataset)
    for god_categ in godLabels:
        cls = god_categ[0]
        id = int(god_categ[1])
        cats = {'supercategory':supercat,
                'id':id,
                'name':cls}
        godCateg.append(cats) 
        
        
    subsetCateg = []
    for org_cat in annoData['categories']:
        org_id = org_cat['id']
        if org_id in id_map:
            god_id = id_map[org_id]
            org_name = org_cat['name']
            if dataset == 'OpenImage':
                org_name = org_name.lower().replace('_',' ')
            cats = {'supercategory':supercat,
                    'id':god_id,
                    'name':name_map[org_name]}
            subsetCateg.append(cats)
    
    
    
    #################################################################################
#     for ix, categ in enumerate(annoData['categories']):
#         src_id = categ['id']
#         if src_id in id_map:
#             categ['id'] = id_map[src_id]
# #             print ("{}==>{}".format(src_id, id_map[src_id]))
# #             print (categ)
#             newCateg.append(categ)
#         else:
#             print ("{} not in GOD".format(categ['name']))
    #################################################################################
    
    print(godCateg[:10])
    print (subsetCateg[:10])
    annoData['categories'] = godCateg
    annoData['subset_categories'] = subsetCateg
    
    #################################################################################
    if osp.exists(outFileName) and not updateAnno:
        godAnnoData = json.load(open(outFileName,'r'))
        annoData['annotations'] = godAnnoData['annotations']
    else: # if not exists(outFileName) or  updateAnno
        newAnnos = []
        for ox, obj in enumerate(annoData['annotations']):
            if obj['category_id'] in id_map:
                src_id = obj['category_id']
                god_id = id_map[obj['category_id']]
                obj['category_id'] = god_id
                newAnnos.append(obj)
                if (ox + 1) % 10000 == 0:
                    print ("{}==>{}".format(src_id, god_id))
    #                 print (obj)
        annoData['annotations'] = newAnnos
    print ("saving to {}".format(outFileName))
    with open(outFileName,'w') as fout:
        json.dump(annoData, fout)
    print ("Done")
    

dataset = COCO_91
outFileName = /media/data/chnxi/GOD/json_annotations/val/COCO_91_val_annotations.json
[{'supercategory': 'GOD_COCO_91', 'id': 1, 'name': 'animal\\amphibian\\frog'}, {'supercategory': 'GOD_COCO_91', 'id': 2, 'name': 'animal\\arthropod\\arachnid\\scorpion'}, {'supercategory': 'GOD_COCO_91', 'id': 3, 'name': 'animal\\arthropod\\arachnid\\spider'}, {'supercategory': 'GOD_COCO_91', 'id': 4, 'name': 'animal\\arthropod\\crustacean\\crab'}, {'supercategory': 'GOD_COCO_91', 'id': 5, 'name': 'animal\\arthropod\\crustacean\\lobster'}, {'supercategory': 'GOD_COCO_91', 'id': 6, 'name': 'animal\\arthropod\\crustacean\\shrimp'}, {'supercategory': 'GOD_COCO_91', 'id': 7, 'name': 'animal\\arthropod\\insect\\ant'}, {'supercategory': 'GOD_COCO_91', 'id': 8, 'name': 'animal\\arthropod\\insect\\bee'}, {'supercategory': 'GOD_COCO_91', 'id': 9, 'name': 'animal\\arthropod\\insect\\butterfly'}, {'supercategory': 'GOD_COCO_91', 'id': 10, 'name': 'animal\\arthropod\\insect\\caterpillar'}]
[{'su